In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
import os
os.environ['JAVA_HOME'] = 'C:\progra~2\Java\jdk1.8.0_231'
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[2] pyspark-shell"

spark = SparkSession.builder.appName("HW2_Submission").getOrCreate()
from pyspark import SparkFiles
from pyspark.sql.functions import *


spark.sparkContext.addFile("https://pbda1.s3.amazonaws.com/n2.csv")


In [18]:
df = spark.read .format("csv").option("header", "true").load(SparkFiles.get("n2.csv"))
df = df.filter(col('SYM').isin(['DJI']))
df.columns
df = df.withColumn("open", df["open"].cast("double"))
df = df.withColumn("close", df["close"].cast("double"))
df = df.withColumn("high", df["high"].cast("double"))
df = df.withColumn("low", df["low"].cast("double"))
df = df.withColumn("volume", df["volume"].cast("double"))
df = df.withColumn('change',df["close"]-df["open"])
df = df.withColumn('label',df["close"])
df = df.withColumn("sentiment_score", df["sentiment_score"].cast("double"))


In [19]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorIndexer,VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler



input_cols = ['open', 'high', 'low', 'close', 'volume','sentiment_score']
assembler = [VectorAssembler(inputCols=input_cols, outputCol="features")]
pipeline = Pipeline(stages=assembler)
df = pipeline.fit(df).transform(df)
# df.show()

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)
scalerModel = scaler.fit(df)
result = scalerModel.transform(df)
# result.show()


from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import col

res = result.select("label","features")
res.printSchema()
(trainingData, testData) = res.randomSplit([0.9, 0.1])

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [27]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="features")

# Train model.  This also runs the indexer.
model = rf.fit(trainingData)

print(type(model))

predictions = model.transform(res)

# Select example rows to display.
predictions.select("prediction", "label", "features")


evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae")
rmse = evaluator.evaluate(predictions)

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
# model.save("stock-model-2.model")

predictions.select("prediction", "label", "features").show(5)

predictions = predictions.withColumn('close',predictions["label"])

<class 'pyspark.ml.regression.RandomForestRegressionModel'>
Root Mean Squared Error (RMSE) on test data = 157.1
+------------------+----------+--------------------+
|        prediction|     label|            features|
+------------------+----------+--------------------+
| 26951.97995809257|27881.7207|[27900.6992,27949...|
| 26951.97995809257|27909.5996|[27987.0508,28010...|
|26969.069520444948|28015.0605|[27839.6797,28035...|
| 26951.97995809257|27677.7891|[27736.0508,27745...|
|26969.069520444948|27649.7793|[27634.6309,27727...|
+------------------+----------+--------------------+
only showing top 5 rows

+----------+--------------------+
|     label|            features|
+----------+--------------------+
|27881.7207|[27900.6992,27949...|
|27909.5996|[27987.0508,28010...|
|28015.0605|[27839.6797,28035...|
|27677.7891|[27736.0508,27745...|
|27649.7793|[27634.6309,27727...|
|27502.8105|[27501.9805,27524...|
|27783.0391|[28109.7402,28109...|
|28051.4102|[28103.1602,28119...|
|   28164.0|